- Need to try to extract training data. 
- need to create a holdout to use to test the predictions with, can be a small holdout 
- there are possibly two different patterns - files from Christine 
- files that have been added since maybe the 4th of March

In [1]:
import glob
from xlrd import open_workbook
import os
import time
import random

In [2]:
excel_files = glob.glob("/Users/ianm/Dropbox/workbench/sage-maker-srm-test/excel-data/*")

In [3]:
len(excel_files)

97

# get files modified on or after the 4th of March

In [4]:
def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    stat = os.stat(path_to_file)
    try:
#         return stat.st_birthtime
        return stat.st_mtime
    except AttributeError:
        # We're probably on Linux. No easy way to get creation dates here,
        # so we'll settle for when its content was last modified.
        return stat.st_mtime

In [5]:
selection_epoch_time = 1551653455 # this is unix time for 3rd of March 2019, 23:00 

In [6]:
post_mar_four_files = []
for file in excel_files:
    mod_time = creation_date(file)
    if mod_time > selection_epoch_time:
        post_mar_four_files.append(file)

In [7]:
%store post_mar_four_files

Stored 'post_mar_four_files' (list)


# get files from a team member

In [8]:
def files_from_team_member(initials, file_list):
    team_member_files = []
    for file in file_list:
        tail_name = file.split("/")[-1]
        if tail_name.split()[0] == initials:
            team_member_files.append(file)
    return team_member_files

# get files from Christine

In [9]:
christine_files = files_from_team_member("CS", excel_files)

# Generic xlrd functions

In [10]:
def get_workbook(filename):
    wb = open_workbook(filename)
    return wb

In [11]:
def get_worksheet(filename, index):
    wb = open_workbook(filename)
    sheet = wb.sheets()[index]
    return sheet

In [12]:
def get_sheet_names(wb):
    return wb.sheet_names() 

In [13]:
def get_col_names(sheet):
    index_row = 0
    col_names = row = sheet.row(index_row)
    return col_names

# Get abstract and titles of selected and rejected articles from Christine files

In [14]:
sheet_names_with_selections = ["contenders", "to email", "names to check for email", "double articles- manual email", "Contenders"]

In [15]:
abstract_col_names = ["AB", "Abstract"]
title_col_names = ["TI", "Title"]

In [16]:
def check_sheet_names_against_validation(sheet_names, validation_list):
    for name in sheet_names:
        if name in validation_list: return True
    return False

In [17]:
def get_col_values_by_col_name(sheet, col_name_candidates):
    """
    the assumption is that we are looking for a colum from one sheet
    but the sheet and colum name might have one of a number of differnet values, but only one of them. 
    """
    col_names = get_col_names(sheet)
    for index, col in enumerate(col_names):
        if col.value in col_name_candidates:
            col_values = sheet.col_values(index)
            return col_values
    return None # if we don't find the column return None

In [18]:
def get_col_values_by_col_name_sheet_name(workbook, sheet_name_candidates, col_name_candidates):
    """
    the assumption is that we are looking for a colum from one sheet
    but the sheet and colum name might have one of a number of differnet values, but only one of them. 
    """
    sheet_names = get_sheet_names(wb)
    for index, sheet_name in enumerate(sheet_names):
        if sheet_name in sheet_name_candidates:
            sheet = wb.sheet_by_name(sheet_name)
            col_values = get_col_values_by_col_name(sheet, col_name_candidates)
            return col_values
    return None # if we don't find the sheet, return None 

In [19]:
all_test_abstracts = []
all_selected_abstracts = []
for file in christine_files:
    wb = get_workbook(file)
    original_sheet_candidates = ["original"]
    original_abstracts = get_col_values_by_col_name_sheet_name(wb, original_sheet_candidates, abstract_col_names)
    selected_abstracts = get_col_values_by_col_name_sheet_name(wb, sheet_names_with_selections, abstract_col_names)
    if original_abstracts and selected_abstracts:
        all_test_abstracts.extend(original_abstracts)
        all_selected_abstracts.extend(selected_abstracts)
        print(len(all_test_abstracts))
        print(len(all_selected_abstracts))
        print("")

209
175

427
369

991
707

1082
720



In [20]:
print(len(list(set(all_selected_abstracts))))

715


# let's remove duplicates.

In [21]:
unique_abstracts= list(set(all_test_abstracts))

In [22]:
unique_selected_abstracts = list(set(all_selected_abstracts))

In [23]:
rejected_abstracts = []
for abstract in unique_abstracts:
    if abstract not in unique_selected_abstracts:
        rejected_abstracts.append(abstract)

In [24]:
print(len(unique_abstracts))
print(len(unique_selected_abstracts))
print(len(rejected_abstracts))

1071
715
356


In [25]:
%store unique_selected_abstracts
%store rejected_abstracts

Stored 'unique_selected_abstracts' (list)
Stored 'rejected_abstracts' (list)


# scramble our lists and get our holdouts

In [154]:
random.shuffle(unique_selected_abstracts)
random.shuffle(rejected_abstracts)

In [157]:
HOLDOUT_NUMBER = 100
holdout_selected_abstracts = unique_selected_abstracts[0:HOLDOUT_NUMBER]
training_selected_abstracts = unique_selected_abstracts[HOLDOUT_NUMBER:]

In [158]:
print(len(unique_selected_abstracts))
print(len(holdout_selected_abstracts))
print(len(training_selected_abstracts))

715
100
615


In [5]:
%store unique_selected_abstracts

UsageError: Unknown variable 'unique_selected_abstracts'


In [30]:
unique_selected_abstracts[0:4]

['',
 'Background: Rheumatic heart disease (RHD) among Ethiopian school children was recently found to be 1.4%. Immigration of the Jewish population from the Gondar region to Israel created an opportunity for further enquiry. Methods: A cross-sectional study of the cardiac status of 113,671 adolescent recruits aged 16-19 years from the northern district of Israel who completed the medical profiling process over a 22-year period. Results: 140 recruits had a history of rheumatic fever (0.12%), although none from an Ethiopian origin (n = 1,719). The prevalence of valvular heart disease clinically and confirmed echocardiographically in Ethiopian recruits was not different from the total population (0.81 and 0.93%, respectively). However, the prevalence was higher in those migrating to Israel in their 13th year or older (2.09%), compared to those migrating at a younger age or born in Israel (0.49%). Conclusion: The Ethiopian teenage Israeli population from Gondar had a high rate of ausculta

we have a couple of duplicate abstracts in our data, boo

we have about 1015 uniqe abstracts, of which 715 have been selected.

need to pick out a set as a holdout to test the trainig algorithim with 

I'm going to pick 100 selected, and 100 non selected abstracts as my holdout, I'm also going to randomize the selection

# Next steps:  

- learn about what format SAGE maker needs for training 
- work though the two lists creating a tagged set of the format that SAGE maker needs  
- take out a 100 item set for both the selected and unselcted ones. 
- shuffle the training set  
- send the training set to SAGE maker   
- send the holdouts to SAGE maker to see how the model preforms  
- repeat the above for titles 